!pip install opencv-python-headless

In [1]:
import pydicom as dicom
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import k3d
import scipy
import cv2

In [2]:
def extract_points(x1, v1, v2, w, h):
    '''
    :param x1: the center point of the slice
    :param v1: one vector in the slice
    :param v2: another vector in the slice
    :param w: width of the slice
    :param h: height of the slice
    :return:
    p1, p2, p3：3 points that can form 2 perpendicular vectors
    '''
    # convert the points and vectors to be np.array
    x1, v1, v2 = np.array(x1), np.array(v1), np.array(v2)
    
    # normalize the 2 given vectors
    v1 = v1 / np.linalg.norm(v1)
    v2 = v2 / np.linalg.norm(v2)

    # calculate the normal vector and normalize the normal vector
    nv = np.cross(v1, v2)
    nv = nv / np.linalg.norm(nv)
    [a,b,c] = nv
    
    # use the unit normal vector to get the 90 degree rotation matrix
    rotate_matrix = np.array([[a**2, a*b-c, a*c+b],[b*a+c, b**2, b*c-a],[c*a-b, c*b+a, c**2]])
    
    # calculate v1's 90 degree rotated vector and normalize it
    v1_p = rotate_matrix@v1.T
    v1_p = v1_p / np.linalg.norm(v1_p)
    
    # get the 3 perpendicular points we want for the slice area
    p1 = x1 + h/2*v1 - w/2*v1_p
    p2 = x1 - h/2*v1 - w/2*v1_p
    p3 = x1 - h/2*v1 + w/2*v1_p
    
    return p1, p2, p3

In [3]:
def slice_area(img, p1, p2, p3, rx, ry):
    '''
    :param img: 3d pixel array
    :param p1,p2,p3: 3 points that can form 2 perpendicular vectors
    :rx: the resolution on x-axis
    :ry: the resolution on y-axis
    :return:
    slicer：2d pixel array
    '''
    # convert all the input points to be np.array
    p1, p2, p3 = np.array(p1), np.array(p2), np.array(p3)
    
    # ratio for linear interpolation and apply them on the line between two points
    t1 = np.linspace(0, 1, rx)[:, None]
    u = t1 * p3[None, :] + (1 - t1) * p2[None, :]
    
    t2 = np.linspace(0, 1, ry)[:, None]
    v = t2 * p1[None, :] + (1 - t2) * p2[None, :]

    # create the position matrix
    p = u[:,None,:] + (v - v[0])[None,:,:]

    # initialize the return slicer
    slicer = np.zeros((rx, ry))
    
    # create a mask that checks if the position is in the bounds
    mask = (p[..., 0] >= 0) & (p[..., 0] < img.shape[0]-1) & (
        p[..., 1] >= 0) & (p[..., 1] < img.shape[1]-1) & (
        p[..., 2] >= 0) & (p[..., 2] < img.shape[2]-1)
    
    #### trilinear interpolation ####
    
    # find the eight points around the position points
    c1 = np.floor(p[mask]).astype(int)
    c2 = c1 + np.array([0,0,1])[None,:]
    c3 = c1 + np.array([0,1,0])[None,:]
    c4 = c1 + np.array([0,1,1])[None,:]
    c5 = c1 + np.array([1,0,0])[None,:]
    c6 = c1 + np.array([1,0,1])[None,:]
    c7 = c1 + np.array([1,1,0])[None,:]
    c8 = c1 + np.array([1,1,1])[None,:]
    
    # the differences
    d = p[mask] - c1
    # get the differences on x-axis
    x_d = d[:,0]
    
    # limit eight points to four points on differences on x-axis
    c00 = (1 - x_d) * img[c1[:, 0], c1[:, 1], c1[:, 2]] + x_d * img[c5[:, 0], c5[:, 1], c5[:, 2]]
    c01 = (1 - x_d) * img[c2[:, 0], c2[:, 1], c2[:, 2]] + x_d * img[c6[:, 0], c6[:, 1], c6[:, 2]]
    c10 = (1 - x_d) * img[c3[:, 0], c3[:, 1], c3[:, 2]] + x_d * img[c7[:, 0], c7[:, 1], c7[:, 2]]
    c11 = (1 - x_d) * img[c4[:, 0], c4[:, 1], c4[:, 2]] + x_d * img[c8[:, 0], c8[:, 1], c8[:, 2]]
    
    # get the differences on y-axis
    y_d = d[:,1]
    
    # limit four points to two points on differences on y-axis
    c0 = (1 - y_d) * c00 + y_d * c10
    c1 = (1 - y_d) * c01 + y_d * c11
    
    # get the differences on z-axis
    z_d = d[:,2]
    
    # limit two points to the final one points on differences on z-axis
    c = (1 - z_d) * c0 + z_d * c1

    # set the final pixel values to its positions in the initialized slicer
    slicer[mask] = c

    return slicer

In [4]:
def plot_2d(s,w,h):
    '''
    :param s: 2d pixel array
    :param w: width of the slice
    :param h: height of the slice
    :return: the image of the 2d pixel array
    '''
    plt.figure(figsize=(8, 8))
    plt.imshow(s.T[::-1], cmap='gray', vmin=0, vmax=s.max(), extent=(0,w,0,h))
    plt.show()

In [5]:
def convert_3d_2d(img,c,v1,v2,w,h,rx,ry):
    '''
    :param img: 3d pixel array
    :param c: the center point of the slice
    :param v1: one vector in the slice
    :param v2: another vector in the slice
    :param w: width of the slice
    :param h: height of the slice
    :rx: the resolution on x-axis
    :ry: the resolution on y-axis
    :return:
    slicer：2d pixel array
    '''
    x1,x2,x3 = extract_points(c,v1,v2,w,h)
    s = slice_area(img,x1,x2,x3,rx,ry)
    plot_2d(s,w,h)
    return s

In [6]:
ds = dicom.dcmread('/workspace/Ting/IM_0013')

In [7]:
plot = k3d.plot()
volume = k3d.factory.volume(ds.pixel_array)
plot += volume
plot.display()

In [8]:
ds.pixel_array.shape

In [9]:
s = convert_3d_2d(ds.pixel_array,[128,200,256],[-1,0,0],[0,0,1],700,300,800,800)
s.shape

In [10]:
s.max()

In [11]:
front_cut = ds.pixel_array[:,200,:]

In [12]:
plt.figure(figsize=(10, 8))
plt.imshow(front_cut, cmap='gray', vmin=0, vmax=255)
plt.show()

In [13]:
# convert the slices to a video
out = cv2.VideoWriter('test.avi',cv2.VideoWriter_fourcc(*'MJPG'),15,(800,800))
for i in np.linspace(0,2*np.pi,60):
    s = convert_3d_2d(ds.pixel_array,[128,200,256],[-np.cos(i),np.sin(i),0],[0,0,1],700,300,800,800)
    s = s.T[::-1]
    s = s[:,:,None]
    s1 = s * np.ones((1,1,3))
    out.write(s1.astype(np.uint8))
out.release()